Button to open Jupyter Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Xx85-O-iRKR6uzPT09wSskZd-RmUg3mh#scrollTo=rMdL5W5Fik4G)

Installing selenium, googletrans

In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.2 MB/s 
     |████████████████████████████████| 358 kB 57.6 MB/s 
     |████████████████████████████████| 138 kB 37.7 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 4.0 MB 47.2 MB/s 
     |████████████████████████████████| 58 kB 1.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 10.3 MB/s 
     |████████████████████████████████| 65 kB 3.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=adf566a0f9c6cb10810624213895e28d3ec8fd6469f4fc5b630fe70a3e130823
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.13.0
    Uninstalling h11-0.13.0:
      Successfully uninstalled h11-0.13.0


Importing Selenium WebDriver and By classes

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By

Importing Chrome-driver and copying it in /bin directory

In [4]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 89.8 MB of archives.
After this operation, 302 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 101.0.4951.64-0ubuntu0.18.04.1 [1,142 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 101.0.4951.64-0ubuntu0.18

Chrome Driver options

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Defining the driver

In [6]:
driver = webdriver.Chrome('chromedriver',options=options)

Products Google sheet url

In [7]:
sheet_url = "https://docs.google.com/spreadsheets/d/1BZSPhk1LDrx8ytywMHWVpCqbm8URTxTJrIRkD7PnGTM/edit#gid=0"
csv_url = sheet_url.replace("/edit#gid=", "/export?format=csv&gid=")

Using Pandas to read URL google sheet

In [8]:
import pandas as pd
url_data = pd.read_csv(csv_url)

In [9]:
url_data.rename(columns={"Unnamed: 0": "SNo."}, inplace=True)
url_data = url_data.set_index('SNo.')
url_data.head()

,id,Asin,country
SNo.,,,
0,1,1015,de
1,2424796,1015,fr
2,2,000004458X,de
3,2424797,000004458X,fr
4,3,1002198,de


Dictionary for Product URLs

In [10]:
url_dic = {}
for i in range(len(url_data)):
    (id, asin, country) = url_data.iloc[i, :].values
    url_dic[id] = {"country" : country, "asin" : asin}

In [11]:
url_dic[1]

{'asin': '1015', 'country': 'de'}

Scraping Function using find_element(s) function to scrape Title, Price, Image URL, Details of a Product

In [12]:
from googletrans import Translator
def prod_info(driver, url):
    driver.get(url)

    page_title  = driver.title
    translator = Translator()
    page_title = translator.translate(page_title).text.split()

    if len(page_title) >= 2:
      if (page_title[-2] == 'not' or page_title[-2] == 'Not') and (page_title[-1] == 'found' or page_title[-1] == 'Found'):
        return url+"\tNot Available", page_title

      else:
        try:
          prod_title = driver.find_element(By.ID, 'productTitle').text
          print(page_title)


          try:
              prod_price_ele = driver.find_element(By.XPATH,"//div[@id ='apex_desktop']")
              prod_price = prod_price_ele.find_element(By.XPATH,".//span[@class='a-offscreen']").get_attribute("innerHTML")
      
          except:
              prod_price_ele = driver.find_element(By.XPATH, "//span[@class='a-color-base']")
              try:
                prod_price = prod_price_ele.find_element(By.TAG_NAME, 'span').text
              except:
                prod_price = prod_price_ele.text


          prod_details = {}
          try:
            try:
                prod_details_ele = driver.find_element(By.XPATH,"//div[@id = 'detailBullets_feature_div']").find_element(By.TAG_NAME,"ul")
                headings = prod_details_ele.find_elements(By.CLASS_NAME,"a-text-bold")
                values = prod_details_ele.find_elements(By.CLASS_NAME,"a-list-item")
                if len(headings) == len(values):
                  for i in range(len(headings)):
                    values[i] = values[i].text[len(headings[i].text):]
                    headings[i] = headings[i].text[:-1]
                    prod_details[headings[i]] = values[i]

            except:
                prod_details_ele = driver.find_element(By.ID,'productDetails_techSpec_section_1')
                headings = prod_details_ele.find_elements(By.TAG_NAME,'th')
                values = prod_details_ele.find_elements(By.TAG_NAME,'td')
                if len(headings) == len(values):
                  for i in range(len(headings)):
                      prod_details[headings[i].text] = values[i].text
          
          except:
            pass


          try:
              prod_img_obj = driver.find_element(By.ID,'img-wrapper')
          except:
              prod_img_obj = driver.find_element(By.ID,'imgTagWrapperId')

          prod_img = prod_img_obj.find_element(By.TAG_NAME,'img')
          prod_img_src = prod_img.get_attribute('src')
          
          print(prod_title, prod_img_src, prod_price, prod_details,'\n')
          return prod_title, prod_img_src, prod_price, prod_details
        
        except:
          return url+"\tNot Available", page_title

    elif len(page_title) <= 1:
      return url+"\tNot Available", page_title

In [13]:
driver.set_page_load_timeout(50)

Scraping Example for Product ID = 2

In [14]:
country = url_dic[2]["country"]
asin = url_dic[2]["asin"]
url = f"https://www.amazon.{country}/dp/{asin}"
try:
      (title, img_url, price, details) = prod_info(driver, url)
except:
      out, pg_title = prod_info(driver, url)
      print(pg_title)
      (title, img_url, price, details) = '','','',''

['Old', 'Spice', 'Shaving', 'Cream', '-', '70', 'g', '(original)', '-', 'Pack', 'of', '2', ':', 'Amazon.de:', 'Health', '&', 'Personal', 'Care']
Old Spice Shaving Cream - 70 g (original) - Pack of 2 https://m.media-amazon.com/images/I/41xHbjdSXJL._SX355_PIbundle-2,TopRight,0,0_SX355SY222SH20_.jpg €8.00 {'Item Package Dimensions L x W x H': '18.3 x 6.7 x 3.9 centimetres', 'Package Weight': '0.18 Kilograms', 'Item Weight': '0.15 Pounds', 'Manufacturer': 'Old Spice', 'Brand Name': 'Old Spice', 'Item Form': 'Creme,Schaum', 'Scent': 'Rasur', 'Item model number': 'SG_000004458X_US', 'Package Dimensions': '18.29 x 6.71 x 3.91 cm; 68.04 Grams', 'ASIN': '000004458X'} 



Storing scraped product information in a list of dictionaries

In [15]:
prod_li = []
for i in url_dic:
    country = url_dic[i]["country"]
    asin = url_dic[i]["asin"]
    url = f"https://www.amazon.{country}/dp/{asin}"
    print(i)
    try:
      (title, img_url, price, details) = prod_info(driver, url)
    except:
      out, pg_title = prod_info(driver, url)
      print(pg_title)
      (title, img_url, price, details) = '','','',''

    if 'out' in locals():
      print(out,"\n")
      del out
    
    prod_li.append({"ID" : "{}".format(i), "Title": title, "Image URL": img_url, "Price": price, "Details": details})

1
['We', 'are', 'sorry!']
https://www.amazon.de/dp/1015	Not Available 

2424796
['Amazon.com', 'page', 'not', 'found']
https://www.amazon.fr/dp/1015	Not Available 

2
['Old', 'Spice', 'Shaving', 'Cream', '-', '70', 'g', '(original)', '-', 'Pack', 'of', '2', ':', 'Amazon.de:', 'Health', '&', 'Personal', 'Care']
Old Spice Shaving Cream - 70 g (original) - Pack of 2 https://m.media-amazon.com/images/I/41xHbjdSXJL._SX355_PIbundle-2,TopRight,0,0_SX355SY222SH20_.jpg €8.00 {'Item Package Dimensions L x W x H': '18.3 x 6.7 x 3.9 centimetres', 'Package Weight': '0.18 Kilograms', 'Item Weight': '0.15 Pounds', 'Manufacturer': 'Old Spice', 'Brand Name': 'Old Spice', 'Item Form': 'Creme,Schaum', 'Scent': 'Rasur', 'Item model number': 'SG_000004458X_US', 'Package Dimensions': '18.29 x 6.71 x 3.91 cm; 68.04 Grams', 'ASIN': '000004458X'} 

2424797
['Our', 'apologies']
https://www.amazon.fr/dp/000004458X	Not Available 

3
['We', 'are', 'sorry!']
https://www.amazon.de/dp/1002198	Not Available 

2424798


Output List

In [18]:
prod_li

[{'Details': '', 'ID': '1', 'Image URL': '', 'Price': '', 'Title': ''},
 {'Details': '', 'ID': '2424796', 'Image URL': '', 'Price': '', 'Title': ''},
 {'Details': {'ASIN': '000004458X',
   'Brand Name': 'Old Spice',
   'Item Form': 'Creme,Schaum',
   'Item Package Dimensions L x W x H': '18.3 x 6.7 x 3.9 centimetres',
   'Item Weight': '0.15 Pounds',
   'Item model number': 'SG_000004458X_US',
   'Manufacturer': 'Old Spice',
   'Package Dimensions': '18.29 x 6.71 x 3.91 cm; 68.04 Grams',
   'Package Weight': '0.18 Kilograms',
   'Scent': 'Rasur'},
  'ID': '2',
  'Image URL': 'https://m.media-amazon.com/images/I/41xHbjdSXJL._SX355_PIbundle-2,TopRight,0,0_SX355SY222SH20_.jpg',
  'Price': '€8.00',
  'Title': 'Old Spice Shaving Cream - 70 g (original) - Pack of 2'},
 {'Details': '', 'ID': '2424797', 'Image URL': '', 'Price': '', 'Title': ''},
 {'Details': '', 'ID': '3', 'Image URL': '', 'Price': '', 'Title': ''},
 {'Details': '', 'ID': '2424798', 'Image URL': '', 'Price': '', 'Title': ''},

Dumping product information to a .json file

In [19]:
import json
products = json.dumps(prod_li, indent=5)
with open('products.json', 'w') as out:
    out.write(products)
out.close()

Installing MySQL connector library

In [20]:
!pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.2 MB 57.9 MB/s 


Products dataframe

In [25]:
products = {"ID" : [], "ImageURL": [], "Price": [], "Details": [], "Title" : []}
for i in prod_li:
  products["ID"].append(i["ID"])
  products["ImageURL"].append(i["Image URL"])
  products["Price"].append(i["Price"])
  products["Details"].append(i["Details"])
  products["Title"].append(i["Title"])

df = pd.DataFrame(data = products)
df = df.set_index('ID')

In [28]:
df.head()

,Image URL,Price,Details,Title
ID,,,,
1,,,,
2424796,,,,
2,https://m.media-amazon.com/images/I/41xHbjdSXJ...,€8.00,{'Item Package Dimensions L x W x H': '18.3 x ...,Old Spice Shaving Cream - 70 g (original) - Pa...
2424797,,,,
3,,,,


 Using mysql connector library for python to dump data locally into a MySQL Database

In [30]:
import mysql.connector
conn = mysql.connector.connect(host = 'localhost', user ='root', password = 'panshu')
cur = conn.cursor()

cur.execute('CREATE DATABASE db;')
cur.execute('SHOW DATABASES;')
conn.close()
conn = mysql.connector.connect(host = 'localhost', user ='root', password = 'panshu', database ='db')
cur = conn.cursor()
cur.execute('create table products (ID varchar(15) not null, Title varchar(255), Price varchar(100), ImageURL TEXT, Details MEDIUMTEXT, PRIMARY KEY(ID));')
conn.commit()

DatabaseError: ignored

Dumping the product dataframe to database 'db' using the existing MySQL connection

In [ ]:
df.to_sql('products', con= conn, if_exists = 'replace', index = True)